In [15]:
import joblib
import pandas as pd
import ipywidgets as widgets
from IPython.display import display
from sentence_transformers import SentenceTransformer, util


juegos = pd.read_csv('databases/juegos_procesados.csv')
libros = pd.read_csv('databases/libros_procesados.csv')
peliculas = pd.read_csv('databases/peliculas_procesados.csv')
sentimientos = pd.read_csv('databases/sentimientos.csv')


In [16]:

embedding_model = SentenceTransformer('sentence-transformers/distiluse-base-multilingual-cased-v1')


In [17]:
def predict_sentiment(text):
    # Step 1: Encode the input text to get embeddings
    text_vectorized = embedding_model.encode([text])
    
    # Step 2: Load the logistic regression model
    try:
        modelo_LR = joblib.load('models/modelo_sentimientos.pkl')
    except:
        print("No se ha encontrado el modelo, correr el notebook de clasificacion")
    
    # Step 3: Predict the sentiment
    sentiment_class = modelo_LR.predict(text_vectorized)[0]
    
    # Step 4: Map the sentiment class to a label
    labels = {0: "Melancolico", 1: "Ni fu Ni fa", 2: "Alegre"}
    sentiment_label = labels[sentiment_class]
    
    return sentiment_label

In [18]:
def get_recommendation(prompt):
    # Step 1: Load the recommendation model
    try:
        modelo_recomendacion = joblib.load('models/modelo_recomendaciones.pkl')
    except:
        print("No se ha encontrado el modelo, correr el notebook de recomendacion")
    
    # Encode descriptions
    juegos_embeddings = modelo_recomendacion.encode(juegos['description'].tolist(), convert_to_tensor=True)
    libros_embeddings = modelo_recomendacion.encode(libros['description'].tolist(), convert_to_tensor=True)
    peliculas_embeddings = modelo_recomendacion.encode(peliculas['description'].tolist(), convert_to_tensor=True)
    
    # Encode user prompt
    user_embedding = modelo_recomendacion.encode(prompt, convert_to_tensor=True)
    
    # Calculate cosine similarities
    coseno_juegos = util.cos_sim(user_embedding, juegos_embeddings)[0]
    coseno_libros = util.cos_sim(user_embedding, libros_embeddings)[0]
    coseno_peliculas = util.cos_sim(user_embedding, peliculas_embeddings)[0]
    
    # Get top recommendations
    index_juego = coseno_juegos.argsort(descending=True)[0].item()
    index_libro = coseno_libros.argsort(descending=True)[0].item()
    index_pelicula = coseno_peliculas.argsort(descending=True)[0].item()
    
    # Extract recommendations
    recomendacion_juego = (juegos['title'].iloc[index_juego], coseno_juegos[index_juego].item())
    recomendacion_libro = (libros['title'].iloc[index_libro], coseno_libros[index_libro].item())
    recomendacion_pelicula = (peliculas['title'].iloc[index_pelicula], coseno_peliculas[index_pelicula].item())
    return recomendacion_juego, recomendacion_libro, recomendacion_pelicula


In [19]:

# Define text areas and buttons
initial_prompt = widgets.Textarea(
    description='Asistente:',
    value='Hola, ¿Cómo estás?',
    layout=widgets.Layout(width='100%'),
    disabled=True
)

followup_prompt = widgets.Textarea(
    description='Asistente:',
    value='Entiendo que estas encerrado y te estas sintiendo reemplazarPorSentimiento. ¿Quieres leer un libro(1), ver una pelicula(2) o jugar un juego(3)?',
    disabled=True,
    layout=widgets.Layout(width='100%')
)

user_initial_input = widgets.Text(
    placeholder='Escriba una respuesta',
    layout=widgets.Layout(width='100%'),
    disabled=False
)

user_followup_input = widgets.Text(
    placeholder='Escribe 1 para libro, 2 para pelicula o 3 para juego',
    layout=widgets.Layout(width='100%'),
    disabled=False
)
user_secondfollowup_input = widgets.Text(
    placeholder='Escribe el tema que te interesa',
    layout=widgets.Layout(width='100%'),
    disabled=False
)

second_followup_prompt = widgets.Textarea(
    description='Asistente:',
    disabled=True,
    layout=widgets.Layout(width='100%')
)

button_initial = widgets.Button(description='Responder')
button_followup = widgets.Button(description='Responder')
button_secondfollowup = widgets.Button(description='Responder')
output = widgets.Output()

recommendation_display = widgets.Textarea(
    description='Asistente:',
    layout=widgets.Layout(width='100%'),
    disabled=True
)



# Callback for initial response
def on_initial_button_clicked(b):
    if(user_initial_input.value == ''):
        return
    with output:
        button_initial.layout.display = 'none'
        output.clear_output()
        sentiment = predict_sentiment(user_initial_input.value)
        followup_prompt.value = followup_prompt.value.replace('reemplazarPorSentimiento', sentiment.lower())
        display(followup_prompt)
        display(user_followup_input)
        display(button_followup)

# Callback for follow-up response
def on_followup_button_clicked(b):
    with output:
        prompt = user_followup_input.value
        if(prompt == '1'):
            second_followup_prompt.value = f"Genial! vamos a buscar libro, sobre que quieres leer?"
        elif(prompt == '2'):
            second_followup_prompt.value = f"Genial! vamos a buscar una película que te guste, sobre que quieres ver?"
        elif(prompt == '3'):
            second_followup_prompt.value = f"Genial! vamos a buscar un juego que te guste, que tema quieres abordar?"
        else:
            user_followup_input.placeholder = 'Por favor ingrese 1, 2 o 3'
            user_followup_input.value = ''
            return
        button_followup.layout.display = 'none'

        display(second_followup_prompt)
        display(user_secondfollowup_input)
        display(button_secondfollowup)

# Callback for follow-up response
def on_secondfollowup_button_clicked(b):
    if(user_secondfollowup_input.value == ''):
        return
    with output:
        button_followup.layout.display = 'none'
        prompt = user_secondfollowup_input.value
        recommendation_display.value = f"Encontre estos resultados para ti: \n"
        display(recommendation_display)

# Bind buttons to callbacks
button_initial.on_click(on_initial_button_clicked)
button_followup.on_click(on_followup_button_clicked)
button_secondfollowup.on_click(on_secondfollowup_button_clicked)

# Display initial prompt and input
display(initial_prompt)
display(user_initial_input)
display(button_initial)
display(output)


Textarea(value='Hola, ¿Cómo estás?', description='Asistente:', disabled=True, layout=Layout(width='100%'))

Text(value='', layout=Layout(width='100%'), placeholder='Escriba una respuesta')

Button(description='Responder', style=ButtonStyle())

Output()